# 4_1_06 Coincidencia entre institución que otorga apoyo público a la pareja/dependientes y la institución donde labora el declarante
| Escenario                                                                                                                                                            | Condición                                                                                | Resultado        |
| -------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------- | ---------------- |
| El beneficio NO es para la pareja/dependiente (es para “Declarante”)                                                                                                 | BeneficiarioPrograma = “DECLARANTE”                                                      | ⚪ **NO_APLICA**  |
| El beneficio es para “Pareja” o “Dependiente económico”, pero institución o nivel faltan                                                                             | Faltan datos clave: institución, nivel, empleo                                           | ⚪ **SIN_DATO**   |
| El beneficio es para pareja/dependiente **y** institución del apoyo coincide con institución laboral del declarante **y además coincide el nivel/orden de gobierno** | `/apoyos/apoyo.institucion == /datosEmpleoCargoComision.nombreEnte` **y** nivel coincide | 🔴 **NO_CUMPLE** |
| El beneficio es para pareja/dependiente pero NO coinciden institución o nivel/orden                                                                                  | Institución ≠ del empleo **o** nivel ≠ del empleo                                        | 🟢 **CUMPLE**    |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "4_1_06_COINCIDENCIA_APOYO_PAREJA_DEP_EMPLEO"

# --- Funciones auxiliares ---

def extraer_valor(doc, path):
    """Extrae strings o números desde un path dinámico."""
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None

        if actual in (None, "", " "):
            return None

        return str(actual).strip()
    except:
        return None


def normalizar_texto(valor):
    if valor is None:
        return None
    return str(valor).strip().upper()


# --- Evaluación principal ---

def evaluar_metrica(doc):

    # -----------------------
    #  Datos de apoyo público
    # -----------------------
    beneficiario = normalizar_texto(
        extraer_valor(doc, "declaracion.intereses.apoyos.apoyo.beneficiarioPrograma")
    )

    institucion_apoyo = normalizar_texto(
        extraer_valor(doc, "declaracion.intereses.apoyos.apoyo.institucion")
    )

    nivel_apoyo = normalizar_texto(
        extraer_valor(doc, "declaracion.intereses.apoyos.apoyo.nivelOrden")
    )

    # -----------------------
    #  Datos del empleo actual
    # -----------------------
    institucion_empleo = normalizar_texto(
        extraer_valor(doc, "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nombreEntePublico")
    )

    nivel_empleo = normalizar_texto(
        extraer_valor(doc, "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelOrden")
    )

    # -----------------------
    #  Lógica institucional
    # -----------------------

    # 1. Si el beneficio es para el declarante → NO_APLICA
    if beneficiario in ("DECLARANTE", "SI", "SERVIDOR", "SERVIDOR PUBLICO"):
        return "NO_APLICA"

    # 2. Si el beneficio NO es para declarante pero faltan datos → SIN_DATO
    if beneficiario is None:
        return "SIN_DATO"

    if institucion_apoyo is None or nivel_apoyo is None:
        return "SIN_DATO"

    if institucion_empleo is None or nivel_empleo is None:
        return "SIN_DATO"

    # 3. Coincidencia total → NO_CUMPLE
    if institucion_apoyo == institucion_empleo and nivel_apoyo == nivel_empleo:
        return "NO_CUMPLE"

    # 4. Caso normal → CUMPLE
    return "CUMPLE"


# --- Procesamiento MongoDB ---

def procesar_metrica_4_1_06():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.intereses.apoyos": 1,
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error doc {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f" > Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}\n")
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass
        print("\nConexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_1_06()


Procesando métrica 4_1_06_COINCIDENCIA_APOYO_PAREJA_DEP_EMPLEO...



c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


 > Procesados 2000 documentos...
 > Procesados 4000 documentos...
 > Procesados 6000 documentos...
 > Procesados 8000 documentos...
 > Procesados 10000 documentos...
 > Procesados 12000 documentos...
 > Procesados 14000 documentos...
 > Procesados 16000 documentos...
 > Procesados 18000 documentos...
 > Procesados 20000 documentos...
 > Procesados 22000 documentos...
 > Procesados 24000 documentos...
 > Procesados 26000 documentos...
 > Procesados 28000 documentos...
 > Procesados 30000 documentos...
 > Procesados 32000 documentos...
 > Procesados 34000 documentos...
